# Import data from Overpass via API

In [1]:
import requests
import json
import pandas as pd
import folium
from shapely.geometry import shape
from shapely.ops import transform
import geopandas as gpd
from pyproj import CRS, Transformer

# Overpass API URL
url = "https://overpass-api.de/api/interpreter"

# OverpassQL-Abfrage mit Bounding Box für Genf
query = """
[out:json][timeout:25];
(
  node["tourism"="attraction"](46.14,6.08,46.23,6.23); // Bounding Box für Genf
  node["tourism"="museum"](46.14,6.08,46.23,6.23);
  node["historic"](46.14,6.08,46.23,6.23);
);
out body;
>;
out skel qt;
"""

# Anfrage an die Overpass API senden
response = requests.get(url, params={"data": query})

# Fehlerbehandlung
if response.status_code == 200:
    data = response.json()
    print("Daten erfolgreich abgerufen!")

    # Speichere die Daten in einer JSON-Datei
    with open("touristenattraktionen_genf.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
else:
    print(f"Fehler bei der Anfrage: {response.status_code}")


Daten erfolgreich abgerufen!


In [2]:
# Schritt 1: Gemeindegrenze von Genf aus Overpass API abrufen
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name="Genève"]->.searchArea;
(
  relation["boundary"="administrative"](area.searchArea);
);
out body;
"""
response = requests.get(overpass_url, params={'data': overpass_query}).json()

# Schritt 2: Koordinaten extrahieren
genf_boundary_coords = []
for element in response['elements']:
    if element['type'] == 'relation':
        for member in element['members']:
            if member['role'] == 'outer' and 'geometry' in member:
                for latlon in member['geometry']:
                    genf_boundary_coords.append([latlon['lat'], latlon['lon']])

# Schritt 3: Polygon erstellen
from shapely.geometry import Polygon
genf_polygon = Polygon(genf_boundary_coords)

# Schritt 4: Buffer um 500 Meter erstellen
# Zuerst müssen wir das Koordinatensystem in UTM (Meter) transformieren
crs_wgs84 = CRS.from_epsg(4326)  # WGS 84
crs_utm = CRS.from_epsg(32632)  # UTM Zone 32N für Genf
transformer = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True)

# Transformiere die Koordinaten des Polygons in UTM
genf_polygon_utm = transform(transformer.transform, genf_polygon)

# Puffer um 500 Meter
genf_buffer_utm = genf_polygon_utm.buffer(500)

# Transformiere den Puffer zurück in WGS84 (Longitude, Latitude)
transformer_reverse = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True)
genf_buffer_wgs84 = transform(transformer_reverse.transform, genf_buffer_utm)

# Schritt 5: Karte erstellen und das Pufferpolygon hinzufügen
map_genf = folium.Map(location=[46.2044, 6.1432], zoom_start=12)

# Pufferpolygon als GeoJSON zur Karte hinzufügen
folium.GeoJson(genf_buffer_wgs84).add_to(map_genf)

# Karte anzeigen
map_genf


In [3]:
# Schritt 1: Gemeindegrenze von Genf aus Overpass API abrufen
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name="Genève"]->.searchArea;
(
  relation["boundary"="administrative"](area.searchArea);
);
out body;
"""
response = requests.get(overpass_url, params={'data': overpass_query}).json()

# Schritt 2: Koordinaten extrahieren
genf_boundary_coords = []
for element in response['elements']:
    if element['type'] == 'relation':
        for member in element['members']:
            if member['role'] == 'outer':
                for latlon in member['geometry']:
                    genf_boundary_coords.append([latlon['lat'], latlon['lon']])

# Schritt 3: Polygon erstellen
from shapely.geometry import Polygon
genf_polygon = Polygon(genf_boundary_coords)

# Schritt 4: Buffer um 500 Meter erstellen
# Zuerst müssen wir das Koordinatensystem in UTM (Meter) transformieren
crs_wgs84 = CRS.from_epsg(4326)  # WGS 84
crs_utm = CRS.from_epsg(32632)  # UTM Zone 32N für Genf
transformer = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True)

# Transformiere die Koordinaten des Polygons in UTM
genf_polygon_utm = transform(transformer.transform, genf_polygon)

# Puffer um 500 Meter
genf_buffer_utm = genf_polygon_utm.buffer(500)

# Transformiere den Puffer zurück in WGS84 (Longitude, Latitude)
transformer_reverse = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True)
genf_buffer_wgs84 = transform(transformer_reverse.transform, genf_buffer_utm)

# Schritt 5: Karte erstellen und das Pufferpolygon sowie die Gemeindegrenze als Linie hinzufügen
map_genf = folium.Map(location=[46.2044, 6.1432], zoom_start=12)

# Hinzufügen der Gemeindegrenze als grüne Linie
folium.PolyLine(
    locations=genf_boundary_coords,  # Gemeindegrenzen-Koordinaten
    color="green",                   # Farbe der Linie
    weight=3,                         # Linienbreite
    opacity=1,                        # Opazität
).add_to(map_genf)

# Pufferpolygon als GeoJSON zur Karte hinzufügen
folium.GeoJson(genf_buffer_wgs84).add_to(map_genf)

# Karte anzeigen (im Notebook oder HTML-Datei speichern)
map_genf


KeyError: 'geometry'

In [30]:


# JSON-Daten laden
with open("touristenattraktionen_genf.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Attraktionen extrahieren
attractions = []
for element in data["elements"]:
    if element["type"] == "node":  # Nur Punkte verwenden
        name = element["tags"].get("name", "Unbenannt")
        lat = element["lat"]
        lon = element["lon"]
        attractions.append({"name": name, "lat": lat, "lon": lon})



# In einer Tabelle darstellen
df_attractions = pd.DataFrame(attractions)
df_attractions.head(1000)


,name,lat,lon
0,Pierre du Niton,46.205673,6.154324
1,Unbenannt,46.204494,6.150513
2,Mur des réformateurs,46.200285,6.145942
3,Jet d'eau,46.207381,6.155889
4,Unbenannt,46.142711,6.126112
...,...,...,...
143,Unbenannt,46.216254,6.151256
144,Pierre à Tasson,46.167192,6.228304
145,Erratic Boulder,46.164167,6.204003
146,Blocs erratiques de l’hôtel-restaurant du Chât...,46.163241,6.201587


In [ ]:
# Interaktive Karte initialisieren
map_genf = folium.Map(location=[46.2044, 6.1432], zoom_start=13)

# Marker hinzufügen
for attraction in attractions:
    folium.Marker(
        location=[attraction["lat"], attraction["lon"]],
        popup=attraction["name"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map_genf)

# Karte im Notebook anzeigen
map_genf
